# <B> SageMaker pileline for Anormaly Detection based on AutoEncoder </B>
* Container: codna_pytorch_p310

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## parameter store 설정

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

## pramamters for tasks

In [4]:
strAccountId = pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"]))
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))
strExecutionRole = pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))
strS3DataPath = pm.get_params(key="-".join([strPrefix, "S3-DATA-PATH"]))

In [5]:
print (f"prefix: {strPrefix}")
print (f"account_id: {strAccountId}")
print (f"defaulut_bucket: {strBucketName}")
print (f"sagemaker_role: {strExecutionRole}")
print (f"s3_data_path: {strS3DataPath}")

prefix: ad-ts
account_id: 419974056037
defaulut_bucket: sm-anomaly-detection-dongjin
sagemaker_role: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
s3_data_path: s3://sm-anomaly-detection-dongjin/data


## 1. Data manipulation and visualization

In [ ]:
import os
import pandas as pd
from utils.util import plot_click_w_fault_and_res, plot_click_w_fault_res_ad, plot_click_w_ad_exp

* load data and derive features

In [ ]:
clicks_1T = pd.read_csv(os.path.join(strS3DataPath, "clicks_1T.csv"), parse_dates=["timestamp"]).set_index("timestamp")
clicks_1T["residual"] = clicks_1T['click'] - clicks_1T['user'] 
clicks_1T["fault"] = pd.read_csv(os.path.join(strS3DataPath, "fault_label_1T.csv"), header=None).values[0] ## label
clicks_1T["time"] = [int(str(time).split(" ")[1].split(":")[0]) for time in clicks_1T.index] ## time variable

In [ ]:
print (f'data shape: {clicks_1T.shape}')
print (f'timestamp min: {clicks_1T.index.min()}, max: {clicks_1T.index.max()}')

* visualization

In [ ]:
plot_click_w_fault_and_res(clicks_1T)

* upload data to s3 and local

In [ ]:
strTrainDataName = "merged_clicks_1T.csv"
clicks_1T.to_csv(os.path.join(strS3DataPath, strTrainDataName), index=True) # to s3
clicks_1T.to_csv(os.path.join("./data", strTrainDataName), index=True) # to local

print (f'train_data_name: {strTrainDataName}')

## 2. Pipeline definition

In [6]:
import os
import argparse
from os import path
from pprint import pprint
from pipeline_config.config import config_handler

from sagemaker.pytorch.estimator import PyTorch
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor
from sagemaker.workflow.retry import StepRetryPolicy, StepExceptionTypeEnum, SageMakerJobExceptionTypeEnum, SageMakerJobStepRetryPolicy

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [7]:
class mlops_pipeline():

    def __init__(self, args):

        self.args = args

        self.strRegionName = self.args.config.get_value("COMMON", "region")
        self.pm = parameter_store(self.strRegionName)
        self._env_setting()        

    def _env_setting(self, ):
        
        self.strPrefix = self.args.config.get_value("COMMON", "prefix")
        self.strExcutionRole = self.args.config.get_value("COMMON", "role")
        self.strBucketName = self.args.config.get_value("COMMON", "bucket")
        self.strModelName = self.args.config.get_value("COMMON", "model_name")
        self.strImageUri = self.args.config.get_value("COMMON", "image_uri")
        self.strPrepImageUri = self.args.config.get_value("COMMON", "image_uri_prep")
        self.strPipelineName = "-".join([self.strPrefix, self.strModelName])
            
        self.cache_config = CacheConfig(
            enable_caching=self.args.config.get_value("PIPELINE", "enable_caching", dtype="boolean"),
            expire_after=self.args.config.get_value("PIPELINE", "expire_after")
        )
        
        self.retry_policies=[                
            # retry when resource limit quota gets exceeded
            SageMakerJobStepRetryPolicy(
                exception_types=[SageMakerJobExceptionTypeEnum.RESOURCE_LIMIT],
                expire_after_mins=180,
                interval_seconds=60,
                backoff_rate=1.0
            ),
        ]
        
        # self.git_config = {
        #     'repo': f'https://{self.pm.get_params(key="-".join([self.strPrefix, "CODE-REPO"]))}',
        #     'branch': 'main',
        #     'username': self.pm.get_params(key="-".join([self.strPrefix, "CODECOMMIT-USERNAME"]), enc=True),
        #     'password': self.pm.get_params(key="-".join([self.strPrefix, "CODECOMMIT-PWD"]), enc=True)
        # }
        
        if self.args.config.get_value("LOCALMODE", "mode", dtype="boolean"): self.pipeline_session = LocalPipelineSession()
        else: self.pipeline_session = PipelineSession()
        
        self.pm.put_params(key="-".join([self.strPrefix, "PIPELINE-NAME"]), value=self.strPipelineName, overwrite=True)
        
        print (f" == Envrionment parameters == ")
        print (f"   SAGEMAKER-ROLE-ARN: {self.strExcutionRole}")
        print (f"   PREFIX: {self.strPrefix}")
        print (f"   BUCKET: {self.strBucketName}")
        print (f"   IMAGE-URI: {self.strImageUri}")
        
    def _step_preprocessing(self, ):
        
        if self.args.config.get_value("LOCALMODE", "mode", dtype="boolean"):
            pipeline_session = LocalPipelineSession()
        else:
            pipeline_session = PipelineSession()
            
        strPrefixPrep = "/opt/ml/processing/"
        strDataPath = self.args.config.get_value("PREPROCESSING", "data_path")
        strTrainDataName = self.args.config.get_value("PREPROCESSING", "data_name")
        
        # network_config로 받으면 된다
        prep_processor = FrameworkProcessor(
            estimator_cls=PyTorch,
            framework_version=self.args.config.get_value("PREPROCESSING", "framework_version"),
            py_version="py310",
            image_uri=None,
            instance_type=self.args.config.get_value("PREPROCESSING", "instance_type"),
            instance_count=self.args.config.get_value("PREPROCESSING", "instance_count", dtype="int"),
            role=self.strExcutionRole,
            base_job_name="preprocessing", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
            sagemaker_session=pipeline_session
        )
        
        step_args = prep_processor.run(
            #job_name="preprocessing", ## 이걸 넣어야 캐시가 작동함, 안그러면 프로세서의 base_job_name 이름뒤에 날짜 시간이 붙어서 캐시 동작 안함
            #git_config=git_config,
            code='preprocessing.py', #소스 디렉토리 안에서 파일 path
            source_dir= "./src/preprocessing", #현재 파일에서 소스 디렉토리 상대경로 # add processing.py and requirements.txt here
            inputs=[
                ProcessingInput(
                    input_name="input-data",
                    source=strDataPath,
                    destination=os.path.join(strPrefixPrep, "input")
                ),
            ],
            outputs=[
                ProcessingOutput(
                    output_name="output-data",
                    source=os.path.join(strPrefixPrep, "output"),
                    destination=os.path.join(
                        "s3://{}".format(self.strBucketName),
                        self.strPipelineName,
                        "preprocessing",
                        "output"
                    )
                ),
            ],
            arguments=[
                "--proc_prefix", strPrefixPrep, \
                "--shingle_size", str(self.args.config.get_value("PREPROCESSING", "shingle_size", dtype="int")), \
                "--train_data_name", strTrainDataName
            ]
        )


        # prep_processor = FrameworkProcessor(
        #     estimator_cls=SKLearn,
        #     image_uri=self.strPrepImageUri,
        #     framework_version=self.args.config.get_value("PREPROCESSING", "framework_version"),
        #     role=self.strExcutionRole,
        #     instance_type=self.args.config.get_value("PREPROCESSING", "instance_type"),
        #     instance_count=self.args.config.get_value("PREPROCESSING", "instance_count", dtype="int"),
        #     base_job_name="preprocessing", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
        #     sagemaker_session=pipeline_session
        # )
            
        # step_args = prep_processor.run(
        #     code='./preprocessing.py', #소스 디렉토리 안에서 파일 path
        #     source_dir="./sources/preprocessing/", #현재 파일에서 소스 디렉토리 상대경로 # add processing.py and requirements.txt here
        #     git_config=self.git_config,
        #     inputs=[
        #         ProcessingInput(
        #             input_name="input",
        #             source=strDataPath,
        #             destination=os.path.join(strPrefixPrep, "input")
        #         ),
        #     ],
        #     outputs=[
        #         ProcessingOutput(
        #             output_name="train-data",
        #             source=os.path.join(strPrefixPrep, "output", "train"),
        #             destination=os.path.join(
        #                 "s3://{}".format(self.strBucketName),
        #                 self.strPipelineName,
        #                 "preprocessing",
        #                 "output",
        #                 "train-data"
        #             ),
        #         ),
        #         ProcessingOutput(
        #             output_name="validation-data",
        #             source=os.path.join(strPrefixPrep, "output", "validation"),
        #             destination=os.path.join(
        #                 "s3://{}".format(self.strBucketName),
        #                 self.strPipelineName,
        #                 "preprocessing",
        #                 "output",
        #                 "validation-data",
        #             ),
        #         ),
        #         ProcessingOutput(
        #             output_name="test-data",
        #             source=os.path.join(strPrefixPrep, "output", "test"),
        #             destination=os.path.join(
        #                 "s3://{}".format(self.strBucketName),
        #                 self.strPipelineName,
        #                 "preprocessing",
        #                 "output",
        #                 "test-data",
        #             ),
        #         )
        #     ],
        #     arguments=["--prefix_prep", strPrefixPrep, "--region", self.strRegionName],
        #     job_name="preprocessing",
        # )
        
        self.preprocessing_process = ProcessingStep(
            name="PreprocessingProcess", ## Processing job이름
            step_args=step_args,
            cache_config=self.cache_config,
        )
        
        print ("  \n== Preprocessing Step ==")
        print ("   \nArgs: ")
        for key, value in self.preprocessing_process.arguments.items():
            print ("===========================")
            print (f'key: {key}')
            pprint (value)
            
        print (type(self.preprocessing_process.properties))
            

#     def _step_training(self, ):
        
#         if self.args.config.get_value("LOCALMODE", "mode", dtype="boolean"):
#             pipeline_session = LocalPipelineSession()
#             pipeline_session.config = {'local': {'local_code': True}}
#         else:
#             pipeline_session = PipelineSession()
                
#         dicHyperparameters = {  
#             "max_depth": "10",
#             "eta": "0.3",
#             "objective": "reg:squarederror",
#             "num_round": "100",
#         }
        
#         self.estimator = XGBoost(
#             entry_point="xgboost_regression.py",
#             source_dir="./sources/train/", #현재 파일에서 소스 디렉토리 상대경로 # add processing.py and requirements.txt here
#             git_config=self.git_config,
#             hyperparameters=dicHyperparameters, ## Contatiner내 env. variable로 들어 감
#             role=self.strExcutionRole,
#             instance_count=self.args.config.get_value("TRAINING", "instance_count", dtype="int"),
#             instance_type=self.args.config.get_value("TRAINING", "instance_type"),
#             framework_version=self.args.config.get_value("TRAINING", "framework_version"),
#             image_uri = self.strImageUri,
#             enable_sagemaker_metrics=True,
#             volume_size=64, ## GB
#             output_path=os.path.join(
#                 "s3://{}".format(self.strBucketName),
#                 self.strPipelineName,
#                 "training",
#                 "output"
#             ),
#             base_job_name="xgboost-train",
#             sagemaker_session=pipeline_session,
#             #metric_definitions=listMetricDefinitions
#         )
        
#         step_training_args = self.estimator.fit(
#             job_name="training",
#             inputs={
#                 "TR": self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["train-data"].S3Output.S3Uri,
#                 "VAL": self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["validation-data"].S3Output.S3Uri,
#                 "TE": self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["test-data"].S3Output.S3Uri,
#             },
#             logs="All",
#         )
          
#         self.training_process = TrainingStep(
#             name="TrainingProcess",
#             step_args=step_training_args,
#             cache_config=self.cache_config,
#             #depends_on=[self.preprocessing_process],
#             retry_policies=self.retry_policies
#         )
        
#         print ("  \n== Training Step ==")
#         print ("   \nArgs: ")
#         for key, value in self.training_process.arguments.items():
#             print ("===========================")
#             print (f'key: {key}')
#             pprint (value)
        
#     def _step_evaluation(self, ):
        
#         #https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks-pytorch.html
        
#         if self.args.config.get_value("LOCALMODE", "mode", dtype="boolean"):
#             pipeline_session = LocalPipelineSession()
#         else:
#             pipeline_session = PipelineSession()
            
#         strPrefixPrep = "/opt/ml/processing/"
        
#         #Initialize the XGBoostProcessor
#         eval_processor = XGBoostProcessor(
#             image_uri=self.strImageUri, 
#             framework_version=self.args.config.get_value("EVALUATION", "framework_version"),
#             role=self.strExcutionRole,
#             instance_type=self.args.config.get_value("EVALUATION", "instance_type"),
#             instance_count=self.args.config.get_value("EVALUATION", "instance_count", dtype="int"),
#             base_job_name='evaluation',
#             sagemaker_session=pipeline_session,
#         )
                
#         self.evaluation_report = PropertyFile(
#             name="EvaluationReport",
#             output_name="evaluation-metrics",
#             path="evaluation-" + self.strModelName +  ".json",
#         )
        
#         step_args = eval_processor.run(
#             job_name="evaluation", # Evaluation job name. If not specified, the processor generates a default job name, based on the base job name and current timestamp.
#                                    # 이걸 넣어야 캐시가 작동함, 안그러면 프로세서의 base_job_name 이름뒤에 날짜 시간이 붙어서 캐시 동작 안함
#             code='evaluation.py', #소스 디렉토리 안에서 파일 path
#             source_dir="./sources/evaluation/", #현재 파일에서 소스 디렉토리 상대경로 # add processing.py and requirements.txt here
#             git_config=self.git_config,
            
#             inputs=[
#                 ProcessingInput(
#                     source=self.training_process.properties.ModelArtifacts.S3ModelArtifacts,
#                     destination=os.path.join(strPrefixPrep, "model") #"/opt/ml/processing/model"
#                 ),
#                 ProcessingInput(
#                     source=self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["test-data"].S3Output.S3Uri,
#                     destination=os.path.join(strPrefixPrep, "test") #"/opt/ml/processing/test"
#                 )
#             ],
#             outputs=[
#                 ProcessingOutput(
#                     output_name="evaluation-metrics",
#                     source=os.path.join(strPrefixPrep, "evaluation"), #"/opt/ml/processing/evaluation",
#                     destination=os.path.join(
#                         "s3://{}".format(self.strBucketName),
#                         self.strPipelineName,
#                         "evaluation",
#                         "evaluation-metrics",
#                     ),
#                 )
#             ],
#             arguments=["--s3_model_path", self.training_process.properties.ModelArtifacts.S3ModelArtifacts, \
#                        "--region", self.strRegionName, "--model_name", self.strModelName, \
#                        "--prefix_eval", strPrefixPrep]
#         )
        
#         self.evaluation_process = ProcessingStep(
#             name="EvaluationProcess", ## Processing job이름들
#             step_args=step_args,
#             #depends_on=[self.preprocessing_process, self.training_process],
#             property_files=[self.evaluation_report],
#             cache_config=self.cache_config,
#             retry_policies=self.retry_policies
#         )
        
#         print ("  \n== Evaluation Step ==")
#         print ("   \nArgs: ")
#         for key, value in self.evaluation_process.arguments.items():
#             print ("===========================")
#             print (f'key: {key}')
#             pprint (value)
        
#     def _step_model_registration(self, ):
        
#         self.strModelPackageGroupName = "-".join(["MPG", self.strPrefix, self.strModelName])
#         self.pm.put_params(key="-".join([self.strPrefix, "MODEL-GROUP-NAME"]), value=self.strModelPackageGroupName, overwrite=True)
                                                                              
#         model_metrics = ModelMetrics(
#             model_statistics=MetricsSource(
#                 s3_uri=Join(
#                     on="/",
#                     values=[
#                         self.evaluation_process.properties.ProcessingOutputConfig.Outputs["evaluation-metrics"].S3Output.S3Uri,
#                         #print (self.evaluation_process.arguments.items())로 확인가능
#                         f"evaluation-{self.strModelName}.json"
#                     ],
#                 ),
#                 content_type="application/json")
#         )
        
#         model = XGBoostModel(
#             entry_point="inference.py",
#             source_dir="./sources/inference/",
#             git_config=self.git_config,
#             framework_version=self.args.config.get_value("MODEL_REGISTER", "framework_version"),
#             code_location=os.path.join(
#                 "s3://",
#                 self.strBucketName,
#                 self.strPipelineName,
#                 "inference",
#                 "model"
#             ),
#             model_data=self.training_process.properties.ModelArtifacts.S3ModelArtifacts,
#             role=self.strExcutionRole,
#             image_uri=self.strImageUri,
#             sagemaker_session=self.pipeline_session,
#         )
        
#         step_args = model.register(
#             content_types=["file-path/raw-bytes", "text/csv"],
#             response_types=["application/json"],
#             inference_instances=self.args.config.get_value("MODEL_REGISTER", "inference_instances", dtype="list"),
#             transform_instances=self.args.config.get_value("MODEL_REGISTER", "transform_instances", dtype="list"),
#             model_package_group_name=self.strModelPackageGroupName,
#             approval_status=self.args.config.get_value("MODEL_REGISTER", "model_approval_status_default"),
#             ## “Approved”, “Rejected”, or “PendingManualApproval” (default: “PendingManualApproval”).
#             model_metrics=model_metrics,
            
#         )
#         self.register_process = ModelStep(
#             name="ModelRegisterProcess",
#             step_args=step_args,
#             depends_on=[self.evaluation_process]
#         )
              
#     def _step_fail(self, ):
            
#         self.fail_process = FailStep(
#             name="ConditionFail",
#             error_message=Join(
#                 on=" ",
#                 values=["Execution failed due to performance threshold"]
#             ),
#         )
        
#     def _step_condition(self, ):
        
#         if self.args.config.get_value("LOCALMODE", "mode", dtype="boolean"):
#             self.pipeline_session = LocalPipelineSession()
#         else:
#             self.pipeline_session = PipelineSession()
        
#         # https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition
#         # 조건문 종류: https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#conditions
        
#         self.condition_acc = ConditionGreaterThanOrEqualTo(
#             left=JsonGet(
#                 step_name=self.evaluation_process.name,
#                 property_file=self.evaluation_report,
#                 json_path="performance_metrics.mse.value" ## evaluation.py에서 json으로 performance를 기록한 대로 한다. 
#                                                                ## 즉, S3에 저장된 evaluation-<model_name>.json 파일안에 있는 값을 적어줘야 한다. 
#             ),
#             right=self.args.config.get_value("CONDITION", "thesh_mse", dtype="float"),
#         )
        
#         self.condition_prec = ConditionGreaterThanOrEqualTo(
#             left=JsonGet(
#                 step_name=self.evaluation_process.name,
#                 property_file=self.evaluation_report,
#                 json_path="performance_metrics.rmse.value" ## evaluation.py에서 json으로 performance를 기록한 대로 한다. 
#                                                            ## 즉, S3에 저장된 evaluation-<model_name>.json 파일안에 있는 값을 적어줘야 한다. 
#             ),
#             right=self.args.config.get_value("CONDITION", "thesh_rmse", dtype="float"),
#         )
        
#         self.condition_process = ConditionStep(
#             name="CheckCondition",
#             display_name="CheckCondition",
#             conditions=[self.condition_acc, self.condition_prec], ## 여러 조건 함께 사용할 수 있음
#             if_steps=[self.register_process],
#             else_steps=[self.fail_process],
#             #depends_on=[self.evaluation_process]
#         )
        
#         print ("  \n== Condition Step ==")
#         print ("   \nArgs: ")
#         for key, value in self.condition_process.arguments.items():
#             print ("===========================")
#             print (f'key: {key}')
#             pprint (value)
        
    def _get_pipeline(self, ):
        
        pipeline = Pipeline(
            name=self.strPipelineName,
            steps=[self.preprocessing_process],
            #steps=[self.preprocessing_process, self.training_process, self.evaluation_process, self.condition_process],
            sagemaker_session=self.pipeline_session
        )

        return pipeline
                            
    def execution(self, ):
         
        self._step_preprocessing()
        # self._step_training()
        # self._step_evaluation()
        # self._step_model_registration()
        # self._step_fail()
        # self._step_condition()
        
        pipeline = self._get_pipeline()
        pipeline.upsert(role_arn=self.strExcutionRole) ## Submit the pipeline definition to the SageMaker Pipelines service 
        execution = pipeline.start()
        desc = execution.describe()
        
#         self.pm.put_params(
#             key="-".join([self.strPrefix, "PIPELINE-ARN"]),
#             value=desc["PipelineArn"],
#             overwrite=True
#         )
        #print (execution.describe())

In [9]:
# strBasePath, strCurrentDir = path.dirname(path.abspath(__file__)), os.getcwd()
# os.chdir(strBasePath)
# print ("==================")
# print (f"  Working Dir: {os.getcwd()}")
# print (f"  You should execute 'mlops_pipeline.py' in 'pipeline' directory'") 
# print ("==================")

parser = argparse.ArgumentParser()
args, _ = parser.parse_known_args()
args.config = config_handler()

print("Received arguments {}".format(args))
os.environ['AWS_DEFAULT_REGION'] = args.config.get_value("COMMON", "region")

pipe_tr = mlops_pipeline(args)
pipe_tr.execution()

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


====== config info. ======
  LOCALMODE: mode:True
  COMMON: prefix:DJ-SM-PIPELINE
  COMMON: region:us-east-1
  COMMON: role:arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
  COMMON: bucket:sm-anomaly-detection-dongjin
  COMMON: model_name:MODEL-1
  COMMON: image_uri_prep:419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-prep:latest
  COMMON: image_uri:419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest
  PIPELINE: enable_caching:True
  PIPELINE: expire_after:T48H
  PREPROCESSING: data_path:s3://sm-anomaly-detection-dongjin/data
  PREPROCESSING: data_name:merged_clicks_1T.csv
  PREPROCESSING: framework_version:2.0.0
  PREPROCESSING: instance_type:ml.m5.xlarge
  PREPROCESSING: instance_count:1
  PREPROCESSING: shingle_size:4
  TRAINING: framework_version:1.5-1
  TRAINING: instance_type:ml.m5.2xlarge
  TRAINING: instance_count:1
  EVALUATION: framework_version:1.5-1
  EVALUATION: instance_type:ml.m5.2xlarge
  EVALUATION: inst

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


 == Envrionment parameters == 
   SAGEMAKER-ROLE-ARN: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
   PREFIX: DJ-SM-PIPELINE
   BUCKET: sm-anomaly-detection-dongjin
   IMAGE-URI: 419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


  
== Preprocessing Step ==
   
Args: 


INFO:sagemaker.processing:Uploaded ./src/preprocessing to s3://sagemaker-us-east-1-419974056037/preprocessing-2024-07-16-14-42-57-922/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-419974056037/preprocessing-2024-07-16-14-42-57-922/source/runproc.sh
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.processing:Uploaded ./src/preprocessing to s3://sagemaker-us-east-1-419974056037/DJ-SM-PIPELINE-MODEL-1/code/fe55da2ec222a058741fa6b3aa5934bd/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-419974056037/DJ-SM-PIPE

key: ProcessingResources
{'ClusterConfig': {'InstanceCount': 1,
                   'InstanceType': 'ml.m5.xlarge',
                   'VolumeSizeInGB': 30}}
key: AppSpecification
{'ContainerArguments': ['--proc_prefix',
                        '/opt/ml/processing/',
                        '--shingle_size',
                        '4',
                        '--train_data_name',
                        'merged_clicks_1T.csv'],
 'ContainerEntrypoint': ['/bin/bash',
                         '/opt/ml/processing/input/entrypoint/runproc.sh'],
 'ImageUri': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.0.0-cpu-py310'}
key: RoleArn
'arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436'
key: ProcessingInputs
[{'AppManaged': False,
  'InputName': 'input-data',
  'S3Input': {'LocalPath': '/opt/ml/processing/input',
              'S3CompressionType': 'None',
              'S3DataDistributionType': 'FullyReplicated',
              'S3DataTyp

INFO:sagemaker.processing:Uploaded ./src/preprocessing to s3://sagemaker-us-east-1-419974056037/DJ-SM-PIPELINE-MODEL-1/code/fe55da2ec222a058741fa6b3aa5934bd/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-419974056037/DJ-SM-PIPELINE-MODEL-1/code/54f0ef6bee583ff9186b762aaf572190/runproc.sh
INFO:sagemaker.local.entities:Starting pipeline step: 'PreprocessingProcess'
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found usin

time="2024-07-16T14:42:59Z" level=warning msg="/tmp/tmpcsb2o3g8/docker-compose.yaml: `version` is obsolete"
time="2024-07-16T14:42:59Z" level=warning msg="a network with name sagemaker-local exists but was not created by compose.\nSet `external: true` to use an existing network"
network sagemaker-local was found but has incorrect label com.docker.compose.network set to ""


In [ ]:
!docker-compose